In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import torch
import torch.nn as nn
import torchvision.transforms as transforms

from concurrent.futures import ThreadPoolExecutor

import rasterio as rio
from rasterio.crs import CRS

import cv2
from affine import Affine
import numpy as np
import glob

from tqdm.notebook import tqdm

from affine import Affine
from itertools import product

from WorldFileUtils import *

Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
def prep(image):
    image = np.asarray(image).astype(np.uint8)
    image = np.uint8(255 - image)
    return image

def prepedges(image):
    image = np.asarray(image).astype(np.uint8)
    image = cv2.Canny(image,50,100)
    image = cv2.dilate(image, np.ones((3,3), np.uint8), iterations=30)
    return np.asarray(image ).astype(np.uint8)

def prepedges_fine(image):
    image = np.asarray(image).astype(np.uint8)
    image = cv2.Canny(image,50,100)
    image = cv2.dilate(image, np.ones((3,3), np.uint8), iterations=5)
    return np.asarray(image ).astype(np.uint8)

# REFERENCE IMAGE FROM QGIS
template = prep(Image.open(r"Harris_Boundary_hollow.png"))[:,:,0]

# TILE INDEX TO TEST ON
image    = prepedges(Image.open(r"masks_black.png"))
image_f  = prepedges_fine(Image.open(r"masks_black.png"))

In [3]:
class ReturnValues:
    def __init__(self, _result, _mask, _rotated_template, _scale, _angle):
        self.result = _result
        self.mask = _mask
        self.rotated_template = _rotated_template
        self.scale = _scale
        self.angle = _angle

class InputValues:
    def __init__(self, scale, angle):
        self.angle = angle
        self.scale = scale
    def __str__(self):
        return f"{self.scale} {self.scale}"
        
def wrapPatternMatch(inputvalue):
    """
    Wrapper function for pattern match to only have one input for multithreading

    Args:
        inputvalue:  Custom class with values for function

    Returns:
        RVs          Custom wrapper class with required values
    """
    return patternMatch(image, template, inputvalue.scale, inputvalue.angle)

def patternMatch(image, template, scale, angle, method=cv2.TM_SQDIFF):
    """
    Pattern matching with rescaling and rotating. 

    Args:
        image:       Image to template match (2D array)
        template:    Template to use for matching (2D array)
        scale:       Scale for which to rescale as percent of original image width
        angle:       Angle to which to rotate template
        method:      Method to use for template matching

    Returns:
        RVs          Custom wrapper class with required values
    """
    
    # RESIZE THE TEMPLATE TO APPROPRIATE SCALE
    scaled_template = cv2.resize(template, None, fx=scale, fy=scale)
    
    # ROTATE THE TEMPLATE TO APPROPRIATE ANGLE
    rotation_matrix = cv2.getRotationMatrix2D((scaled_template.shape[1] / 2, scaled_template.shape[0] / 2), angle, 1.0)
    rotated_template = cv2.warpAffine(scaled_template, rotation_matrix, (scaled_template.shape[1], scaled_template.shape[0]))
    
    # CREATE MASK TO NOT COUNT ZEROS IN TEMPLATE
    mask = np.where(rotated_template == 0, 0, 255).astype(np.uint8)
    
    # RUN PATTERNMATCHING
    result = cv2.matchTemplate(image, rotated_template, method, mask=mask)    
    return ReturnValues(result, mask,rotated_template, scale, angle)

def postprocess_results(result_list, scales, angles, opt_max):
    """
    This converts the results from the patternmatching workflow to 
    actionable information as described below

    Args:
        result_list: List of 2D arrays containing results from pattern matching
        scales:      1D array of scales used for current pattern matching 
        angles:      1D array of angles used for current pattern matching 
        opt_max:     Flag indicating whether best is Max or Min. False = Min

    Returns:
        x:           Best predicted X for pattern matching
        y:           Best predicted Y for pattern matching
        br:          Tuple containing (X, Y) coordinates of predicted bottom right corner
        rf:          Predicted rescale factor
        ang:         Predicted angle
    """
    
    # FLATTEN ALL ARRAYS TO CALCULATE CUTOFF VALUES FOR CANDIATES
    elem_list = [x.flatten() for x in result_list]
    elem_list = np.hstack(elem_list)
    
    # CUTOFF VALUES FOR CANDIDATES
    if opt_max:
        thresh = np.percentile(elem_list, 99.9)
        loc_list = [x > thresh for x in result_list]
    else:
        thresh = np.percentile(elem_list, 0.1)
        loc_list = [x < thresh for x in result_list]
        
    # VOTES ON DIFFERENT PATTERN MATCHING AT DIFFERENT VALUES
    votes = np.array([np.count_nonzero(x) for x in loc_list])
    
    rf  = np.sum(votes * scales) / np.sum(votes)
    ang = np.sum(votes * angles) / np.sum(votes)
    
    # LIST OF X AND Y VALUES
    x_list = np.hstack([np.where(x)[0] for x in loc_list])
    y_list = np.hstack([np.where(x)[1] for x in loc_list])
    
    # RETURN MEDIAN OF VALUES
    x = int(np.median(x_list))
    y = int(np.median(y_list))
    
    # BOTTOM RIGHT VALUE
    br = (int(y + template.shape[1] * rf), int(x + template.shape[0] * rf))
    
    return x, y, br, rf, ang

In [4]:
def SearchOptimalMatch(initial_guess, perturbance, angle=0, change_ang=None, retall=True, rngpert=2):
    
    # WHICH ANGLES AND SCALES TO SEARCH
    if change_ang is None:
        scales = np.arange(initial_guess-perturbance, initial_guess+perturbance+1e-5, perturbance/rngpert)
        inputs = [InputValues(scale, angle) for scale in scales]
        angles = [0]
    else:
        scales = np.arange(initial_guess-perturbance, initial_guess+perturbance+1e-5, perturbance/rngpert)
        angles = np.arange(angle-change_ang, angle+change_ang+1e-5, change_ang/2)
        inputs = [InputValues(scale, angle) for scale, angle in product(scales, angles)]
        scales = [IV.scale for IV in inputs]
        angles  = [IV.angle for IV in inputs]
                
    # INITIALIZE BEST MATCH VARIABLES
    best_match_scale = 1.0
    best_match_angle = 0.0
    
    # IS BEST MAX OR MIN? FALSE = MIN
    opt_max = False
    if opt_max:
        best_match_value = -1 * np.inf
    else:
        best_match_value = np.inf
        
    result_list = list()
    best_loc_list = list()
    rect_corner_list = list()

    with ThreadPoolExecutor(max_workers=8) as executor:
        for retvalue in tqdm(executor.map(wrapPatternMatch, inputs), total=len(inputs)):

            mask = retvalue.mask
            result = retvalue.result
            rotated_template = retvalue.rotated_template
            scale = retvalue.scale
            angle = retvalue.angle
            if not opt_max:
                result = np.sqrt(result / np.count_nonzero(mask))

            result_list.append(result)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

            if opt_max:
                best_val = max_val
                best_loc = max_loc
                bottom_right = (best_loc[0] + int(rotated_template.shape[1]), best_loc[1] + int(rotated_template.shape[0]))

                best_loc_list.append(best_loc)
                rect_corner_list.append(bottom_right)

            else:
                best_val = min_val
                best_loc = min_loc
                bottom_right = (best_loc[0] + int(rotated_template.shape[1]), best_loc[1] + int(rotated_template.shape[0]))
                best_loc_list.append(best_loc)
                rect_corner_list.append(bottom_right)            

            print(scale, angle, best_val)
        
        postprocess = postprocess_results(result_list, scales, angles, opt_max)
    
    if not retall:
        return postprocess[3]
    return postprocess

In [5]:
curr_guess = 0.8
perturbations = [0.1, 0.05, 0.025, 0.0125]

for i, pert in enumerate(perturbations):
    x, y, br, rescale_factor, angle = SearchOptimalMatch(curr_guess, pert)
    curr_guess = rescale_factor
    print(f"New Guess: {curr_guess} {angle}")
    
ang_search    = 1
x, y, br, rescale_factor, angle = SearchOptimalMatch(curr_guess, perturbations[-1], change_ang=ang_search)
curr_guess = rescale_factor

print(f"New Guess: {curr_guess} {angle}")
print(rescale_factor)

  0%|          | 0/5 [00:00<?, ?it/s]

0.7000000000000001 0 216.62223399738718
0.7500000000000001 0 215.6942329081225
0.8000000000000002 0 216.78464493077408
0.8500000000000002 0 204.41115459919004
0.9000000000000002 0 208.84378380878118
New Guess: 0.868405879365459 0.0


  0%|          | 0/5 [00:00<?, ?it/s]

0.8184058793654589 0 215.9508292292257
0.843405879365459 0 206.19018216747364
0.868405879365459 0 180.93880574374046
0.893405879365459 0 206.6386741173798
0.918405879365459 0 212.63262653488525
New Guess: 0.868405879365459 0.0


  0%|          | 0/5 [00:00<?, ?it/s]

0.843405879365459 0 206.19018216747364
0.8559058793654589 0 203.9508696244984
0.8684058793654589 0 180.93880574374046
0.8809058793654588 0 190.5873356924205
0.8934058793654588 0 206.6386741173798
New Guess: 0.8705303580813294 0.0


  0%|          | 0/5 [00:00<?, ?it/s]

0.8580303580813294 0 201.06806545887088
0.8642803580813294 0 183.98855551085148
0.8705303580813294 0 180.74920454884904
0.8767803580813294 0 186.84215725664637
0.8830303580813293 0 194.825116455651
New Guess: 0.8698789292739293 0.0


  0%|          | 0/25 [00:00<?, ?it/s]

0.8573789292739293 -1.0 194.02931054950855
0.8573789292739293 -0.5 192.75474811169656
0.8573789292739293 0.0 202.6500511952986
0.8573789292739293 0.5 193.79005099742318
0.8573789292739293 1.0 192.51298488591968
0.8636289292739293 -1.0 188.68351020977087
0.8636289292739293 -0.5 184.72704950800684
0.8636289292739293 0.0 185.67588414314156
0.8636289292739293 0.5 183.64715840329973
0.8636289292739293 1.0 185.70074082826682
0.8698789292739293 -1.0 181.85124086522555
0.8698789292739293 -0.5 178.52225485365565
0.8698789292739293 0.0 180.52628770938796
0.8698789292739293 0.5 180.2149232736075
0.8698789292739293 1.0 186.8776176199957
0.8761289292739293 -1.0 178.10291682855438
0.8761289292739293 -0.5 175.44745888798573
0.8761289292739293 0.0 186.22529746667593
0.8761289292739293 0.5 188.7176209551062
0.8761289292739293 1.0 195.3469347984824
0.8823789292739292 -1.0 179.60170172529058
0.8823789292739292 -0.5 183.74141713414875
0.8823789292739292 0.0 193.44052215206565
0.8823789292739292 0.5 194.85

In [6]:
three_band = np.dstack([image, image, image])
_ = cv2.rectangle(three_band, (y, x), br, (0, 255, 0), 5)

In [11]:
tfw_file        = "Harris_Boundary_hollow.pgw"
template_affine = get_affine_from_geotransform(get_geotransform_from_tfw(tfw_file)[0])
calc_affine     = Affine(1 / rescale_factor, 0, -y, 0, 1 / rescale_factor, -x) * Affine.rotation(angle)

In [12]:
# out_affine = combineAffine(template_affine, calc_affine)
out_affine = template_affine * calc_affine
write_world_file_from_affine(out_affine, "data/TileIndices/48201CIND0_0992.tfw")